In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

raw_data = pd.read_csv("CompleteDataSet_training_competition_1500.csv", sep=",")

data = raw_data[['WristAccelerometer x-axis', 'WristAccelerometer y-axis', 'WristAccelerometer z-axis', 'Activity']]
av_data = raw_data[['WristAngularVelocity x-axis', 'WristAngularVelocity y-axis', 'WristAngularVelocity z-axis']]

n_step = 1500
n_batch = int(data.shape[0] / n_step)

# variables used in the paper
k = 15
t = 3
K = 300

xyz = 3

data = data.as_matrix()
av_data = av_data.as_matrix()
calculate = np.zeros([n_batch * int(k / t), 43 + 1])

# for each data
for i in range(n_batch):
    
    # for each sliding window
    for j in range(int(k / t)):
        calculate[int(k / t) * i + j][43] = data[i * n_step][xyz]
                
        x = np.zeros([K])
        y = np.zeros([K])
        z = np.zeros([K])
        a = np.zeros([K])
        sa = 0        
        se = np.zeros([xyz])
        fft_x = np.zeros([K])
        fft_y = np.zeros([K])
        fft_z = np.zeros([K])
        fft_a = np.zeros([K])
        sqr_x = np.zeros([K])
        sqr_y = np.zeros([K])
        sqr_z = np.zeros([K])
        sqr_a = np.zeros([K])
        av = np.zeros([xyz+1])
        psd_x = 0
        psd_y = 0
        psd_z = 0
        psd_a = 0
        entropy = np.zeros([xyz+1])
        corr = np.zeros((xyz, xyz))

        for m in range(K):                
            x[m] = data[i * n_step + j * K + m][0]
            y[m] = data[i * n_step + j * K + m][1]
            z[m] = data[i * n_step + j * K + m][2]
            a[m] = np.sqrt(pow(data[i * n_step + j * K + m][0], 2) + pow(data[i * n_step + j * K + m][1], 2) + pow(data[i * n_step + j * K + m][2], 2))
            sa += (np.abs(x[m]) + np.abs(y[m]) + np.abs(z[m]))
            av[0] += av_data[i * n_step + j * K + m][0]
            av[1] += av_data[i * n_step + j * K + m][1]
            av[2] += av_data[i * n_step + j * K + m][2]
            av[3] += np.sqrt(pow(av_data[i * n_step + j * K + m][0], 2) + pow(av_data[i * n_step + j * K + m][1], 2) + pow(av_data[i * n_step + j * K + m][2], 2))
        
        fft_x = np.fft.fft(x)
        fft_y = np.fft.fft(y)
        fft_z = np.fft.fft(z)
        fft_a = np.fft.fft(a)
        for m in range(K):
            se[0] += pow(fft_x[m], 2)
            se[1] += pow(fft_y[m], 2)
            se[2] += pow(fft_z[m], 2)
            
            sqr_x[m] = np.sqrt(fft_x[m])
            sqr_y[m] = np.sqrt(fft_y[m])
            sqr_z[m] = np.sqrt(fft_z[m])
            sqr_a[m] = np.sqrt(fft_a[m])
      
        psd_x = np.mean(sqr_x)
        psd_y = np.mean(sqr_y)
        psd_z = np.mean(sqr_z)
        psd_a = np.mean(sqr_a)
        entropy[0] = psd_x * np.log2(psd_x)
        entropy[1] = psd_y * np.log2(psd_y)
        entropy[2] = psd_z * np.log2(psd_z)
        entropy[3] = psd_a * np.log2(psd_a)
        
        lst = [x, y, z]
        df = pd.DataFrame(lst).T
        corr = df.corr(method = 'pearson')
        
        calculate[int(k / t) * i + j][0] = np.std(x)
        calculate[int(k / t) * i + j][1] = np.std(y)
        calculate[int(k / t) * i + j][2] = np.std(z)
        calculate[int(k / t) * i + j][3] = np.std(a)
        calculate[int(k / t) * i + j][4] = np.mean(x)
        calculate[int(k / t) * i + j][5] = np.mean(y)
        calculate[int(k / t) * i + j][6] = np.mean(z)
        calculate[int(k / t) * i + j][7] = np.mean(a)
        calculate[int(k / t) * i + j][8] = max(x)
        calculate[int(k / t) * i + j][9] = max(y)
        calculate[int(k / t) * i + j][10] = max(z)
        calculate[int(k / t) * i + j][11] = max(a)
        calculate[int(k / t) * i + j][12] = min(x)
        calculate[int(k / t) * i + j][13] = min(y)
        calculate[int(k / t) * i + j][14] = min(z)
        calculate[int(k / t) * i + j][15] = min(a)
        calculate[int(k / t) * i + j][16] = sa
        calculate[int(k / t) * i + j][17] = np.median(x)
        calculate[int(k / t) * i + j][18] = np.median(y)
        calculate[int(k / t) * i + j][19] = np.median(z)
        calculate[int(k / t) * i + j][20] = np.median(a)
        calculate[int(k / t) * i + j][21] = np.percentile(x, 25)
        calculate[int(k / t) * i + j][22] = np.percentile(y, 25)
        calculate[int(k / t) * i + j][23] = np.percentile(z, 25)
        calculate[int(k / t) * i + j][24] = np.percentile(a, 25)
        calculate[int(k / t) * i + j][25] = np.percentile(x, 75)
        calculate[int(k / t) * i + j][26] = np.percentile(y, 75)
        calculate[int(k / t) * i + j][27] = np.percentile(z, 75)
        calculate[int(k / t) * i + j][28] = np.percentile(a, 75)
        calculate[int(k / t) * i + j][29] = se[0]
        calculate[int(k / t) * i + j][30] = se[1]
        calculate[int(k / t) * i + j][31] = se[2]
        calculate[int(k / t) * i + j][32] = av[0] / K
        calculate[int(k / t) * i + j][33] = av[1] / K
        calculate[int(k / t) * i + j][34] = av[2] / K
        calculate[int(k / t) * i + j][35] = av[3] / K
        calculate[int(k / t) * i + j][36] = entropy[0]
        calculate[int(k / t) * i + j][37] = entropy[1]
        calculate[int(k / t) * i + j][38] = entropy[2]
        calculate[int(k / t) * i + j][39] = entropy[3]
        calculate[int(k / t) * i + j][40] = corr[0][1]
        calculate[int(k / t) * i + j][41] = corr[1][2]
        calculate[int(k / t) * i + j][42] = corr[0][2]
        
data = pd.DataFrame(calculate)

data.to_csv("./dataset/training_data_LSTM_FS_3_wrist.csv", sep=",", header=False, index=False)